# Portfolio Optimization

Portfolio Optimization is used for risk-averse investors to construct portfolios to optimize or maximize expected return based on a given level of market risk, emphasizing that risk is an inherent part of higher reward

This notebook:
1. Runs an example Monte Carlo Simulation for an optimal portfolio with resulting returns 
2. Creates an Efficient Frontier which is used to identify a set of optimal portfolios that offers the highest expected return for a defined level of risk or the lowest risk for a given level of expected return

Monte Carlo simulations are used by analyst to determine the expected value and optimal distribution of a portfolio. 

In [2]:
import numpy as np
import pandas as pd
import hvplot.pandas  # noqa
import yfinance as yf
import numpy as np
import requests
import json
import pandas as pd
import datetime as dt
import holoviews as hv

In [3]:
#Ask user how much is the allocation for Equity Allocation
# Check the input 
while True:
    try:
        equity_allocation = float(input('Please enter the amount of capital you wish to invest :'))
        break
    except:
        print('Please enter a numerical value.')

In [4]:
# Prompt the user how many years of historical data they wish to pull for the analysis
# Check the input 
while True:
    try:
        num_years = int(input('Please enter the number of years (1-10) of historical data that you wish to retrieve for the analysis :'))
        break
    except:
        print('Please enter a numerical value.'
             )

In [5]:
api_pull = {'commodities' : ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']}

In [6]:
data = {}
for asset_class, tickers in api_pull.items():
    data[asset_class] = {}
    for ticker in tickers:
        data[asset_class][ticker] = yf.Ticker(ticker).history(period=f"{num_years}y")

In [7]:
# Review pulled data:
data 

{'commodities': {'GC=F':                                   Open         High          Low        Close  \
  Date                                                                            
  2020-02-18 00:00:00-05:00  1579.800049  1604.300049  1579.800049  1600.000000   
  2020-02-19 00:00:00-05:00  1600.599976  1610.500000  1599.699951  1607.500000   
  2020-02-20 00:00:00-05:00  1606.500000  1621.000000  1603.000000  1616.599976   
  2020-02-21 00:00:00-05:00  1619.300049  1645.000000  1619.300049  1644.599976   
  2020-02-24 00:00:00-05:00  1657.000000  1686.599976  1650.000000  1672.400024   
  ...                                ...          ...          ...          ...   
  2023-02-09 00:00:00-05:00  1875.300049  1884.599976  1859.800049  1866.199951   
  2023-02-10 00:00:00-05:00  1861.599976  1863.500000  1852.400024  1862.800049   
  2023-02-13 00:00:00-05:00  1859.000000  1861.000000  1850.000000  1851.900024   
  2023-02-14 00:00:00-05:00  1854.099976  1862.000000  1846.1999

In [8]:
df_commodities = pd.DataFrame({ticker: data['commodities'][ticker]['Close'] for ticker in api_pull['commodities']})

In [9]:
df_commodities.head()

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2020-02-18 00:00:00-05:00,1600.000000,18.131001,52.049999,2.6100,446.100006,892.25,139.300003,485.75,67.870003,1343.0
2020-02-19 00:00:00-05:00,1607.500000,18.292000,53.290001,2.6125,460.299988,897.25,140.774994,479.75,68.480003,1342.5
2020-02-20 00:00:00-05:00,1616.599976,18.309000,53.779999,2.5950,463.000000,892.75,140.800003,473.75,68.750000,1343.5
2020-02-21 00:00:00-05:00,1644.599976,18.521000,53.380001,2.6180,460.100006,890.50,140.199997,468.50,68.930000,1338.5
2020-02-24 00:00:00-05:00,1672.400024,18.868000,51.430000,2.5925,443.100006,874.25,136.024994,452.25,67.529999,1337.0


In [10]:
df_commodities = df_commodities.fillna(method='ffill')
df_commodities

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2020-02-18 00:00:00-05:00,1600.000000,18.131001,52.049999,2.6100,446.100006,892.25,139.300003,485.75,67.870003,1343.000
2020-02-19 00:00:00-05:00,1607.500000,18.292000,53.290001,2.6125,460.299988,897.25,140.774994,479.75,68.480003,1342.500
2020-02-20 00:00:00-05:00,1616.599976,18.309000,53.779999,2.5950,463.000000,892.75,140.800003,473.75,68.750000,1343.500
2020-02-21 00:00:00-05:00,1644.599976,18.521000,53.380001,2.6180,460.100006,890.50,140.199997,468.50,68.930000,1338.500
2020-02-24 00:00:00-05:00,1672.400024,18.868000,51.430000,2.5925,443.100006,874.25,136.024994,452.25,67.529999,1337.000
...,...,...,...,...,...,...,...,...,...,...
2023-02-09 00:00:00-05:00,1866.199951,22.097000,78.059998,4.1055,433.600006,1519.25,186.824997,879.00,85.500000,1811.500
2023-02-10 00:00:00-05:00,1862.800049,22.034000,79.720001,4.0275,419.600006,1542.50,186.399994,909.00,85.269997,1808.500
2023-02-13 00:00:00-05:00,1851.900024,21.818001,80.139999,4.0870,414.100006,1542.75,187.199997,912.25,85.639999,1791.000


In [11]:
portfolio_data = pd.concat([df_commodities], axis=1)
portfolio_data

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2020-02-18 00:00:00-05:00,1600.000000,18.131001,52.049999,2.6100,446.100006,892.25,139.300003,485.75,67.870003,1343.000
2020-02-19 00:00:00-05:00,1607.500000,18.292000,53.290001,2.6125,460.299988,897.25,140.774994,479.75,68.480003,1342.500
2020-02-20 00:00:00-05:00,1616.599976,18.309000,53.779999,2.5950,463.000000,892.75,140.800003,473.75,68.750000,1343.500
2020-02-21 00:00:00-05:00,1644.599976,18.521000,53.380001,2.6180,460.100006,890.50,140.199997,468.50,68.930000,1338.500
2020-02-24 00:00:00-05:00,1672.400024,18.868000,51.430000,2.5925,443.100006,874.25,136.024994,452.25,67.529999,1337.000
...,...,...,...,...,...,...,...,...,...,...
2023-02-09 00:00:00-05:00,1866.199951,22.097000,78.059998,4.1055,433.600006,1519.25,186.824997,879.00,85.500000,1811.500
2023-02-10 00:00:00-05:00,1862.800049,22.034000,79.720001,4.0275,419.600006,1542.50,186.399994,909.00,85.269997,1808.500
2023-02-13 00:00:00-05:00,1851.900024,21.818001,80.139999,4.0870,414.100006,1542.75,187.199997,912.25,85.639999,1791.000


In [12]:
portfolio_data_daily_ret = portfolio_data.pct_change(1).dropna()
portfolio_data_daily_ret.head()

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2020-02-19 00:00:00-05:00,0.004687,0.008880,0.023823,0.000958,0.031831,0.005604,0.010589,-0.012352,0.008988,-0.000372
2020-02-20 00:00:00-05:00,0.005661,0.000929,0.009195,-0.006699,0.005866,-0.005015,0.000178,-0.012507,0.003943,0.000745
2020-02-21 00:00:00-05:00,0.017320,0.011579,-0.007438,0.008863,-0.006263,-0.002520,-0.004261,-0.011082,0.002618,-0.003722
2020-02-24 00:00:00-05:00,0.016904,0.018735,-0.036531,-0.009740,-0.036948,-0.018248,-0.029779,-0.034685,-0.020310,-0.001121
2020-02-25 00:00:00-05:00,-0.015248,-0.036305,-0.029749,0.001543,-0.017603,0.005433,-0.022422,0.006081,-0.017770,-0.000374


In [13]:
mean_daily_ret = portfolio_data.pct_change(1).mean()
mean_daily_ret

GC=F     0.000253
SI=F     0.000512
CL=F    -0.003545
HG=F     0.000710
LBS=F    0.000868
ZS=F     0.000824
GF=F     0.000470
KE=F     0.001051
CT=F     0.000514
ZR=F    -0.000753
dtype: float64

In [14]:
portfolio_data.pct_change(1).corr()

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
GC=F,1.000000,0.788243,0.046100,0.279670,0.045511,0.166456,0.031139,0.130877,0.117656,0.008894
SI=F,0.788243,1.000000,0.062571,0.356474,0.065646,0.183264,0.060222,0.163839,0.153914,0.000333
CL=F,0.046100,0.062571,1.000000,0.132470,0.065788,0.082708,0.042546,0.006383,0.048964,-0.003934
HG=F,0.279670,0.356474,0.132470,1.000000,0.091574,0.270652,0.040222,0.171977,0.282433,0.070527
LBS=F,0.045511,0.065646,0.065788,0.091574,1.000000,0.075284,0.036001,0.052643,0.037793,-0.065368
ZS=F,0.166456,0.183264,0.082708,0.270652,0.075284,1.000000,-0.069360,0.389813,0.114651,0.063357
GF=F,0.031139,0.060222,0.042546,0.040222,0.036001,-0.069360,1.000000,-0.151913,0.028650,-0.046300
KE=F,0.130877,0.163839,0.006383,0.171977,0.052643,0.389813,-0.151913,1.000000,0.189720,0.086098
CT=F,0.117656,0.153914,0.048964,0.282433,0.037793,0.114651,0.028650,0.189720,1.000000,0.082668
ZR=F,0.008894,0.000333,-0.003934,0.070527,-0.065368,0.063357,-0.046300,0.086098,0.082668,1.000000


In [15]:
portfolio_data.head()

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2020-02-18 00:00:00-05:00,1600.000000,18.131001,52.049999,2.6100,446.100006,892.25,139.300003,485.75,67.870003,1343.0
2020-02-19 00:00:00-05:00,1607.500000,18.292000,53.290001,2.6125,460.299988,897.25,140.774994,479.75,68.480003,1342.5
2020-02-20 00:00:00-05:00,1616.599976,18.309000,53.779999,2.5950,463.000000,892.75,140.800003,473.75,68.750000,1343.5
2020-02-21 00:00:00-05:00,1644.599976,18.521000,53.380001,2.6180,460.100006,890.50,140.199997,468.50,68.930000,1338.5
2020-02-24 00:00:00-05:00,1672.400024,18.868000,51.430000,2.5925,443.100006,874.25,136.024994,452.25,67.529999,1337.0


In [16]:
#portfolio_data.replace(to_replace = 0, value = np.NAN, inplace=True)
portfolio_data = portfolio_data
log_ret = np.log(portfolio_data/portfolio_data.shift(1))
log_ret.head()

/Users/alphonsologan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2020-02-18 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-19 00:00:00-05:00,0.004677,0.008841,0.023544,0.000957,0.031335,0.005588,0.010533,-0.012429,0.008948,-0.000372
2020-02-20 00:00:00-05:00,0.005645,0.000929,0.009153,-0.006721,0.005849,-0.005028,0.000178,-0.012585,0.003935,0.000745
2020-02-21 00:00:00-05:00,0.017172,0.011512,-0.007465,0.008824,-0.006283,-0.002523,-0.004271,-0.011144,0.002615,-0.003729
2020-02-24 00:00:00-05:00,0.016763,0.018562,-0.037214,-0.009788,-0.037648,-0.018417,-0.030231,-0.035301,-0.020520,-0.001121


In [17]:
log_ret.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
GC=F,758.0,0.000192,0.011079,-0.051069,-0.004933,0.000523,0.006056,0.057775
SI=F,758.0,0.000239,0.023399,-0.123854,-0.011125,0.000775,0.011484,0.088800
CL=F,756.0,0.001349,0.040370,-0.282206,-0.014968,0.003105,0.018207,0.319634
HG=F,758.0,0.000577,0.016279,-0.069286,-0.009318,0.000700,0.011449,0.071988
LBS=F,758.0,-0.000048,0.043141,-0.407641,-0.022423,0.002071,0.025567,0.267733
ZS=F,758.0,0.000709,0.015235,-0.110917,-0.007035,0.001573,0.009339,0.064263
GF=F,758.0,0.000389,0.012676,-0.081190,-0.005572,0.000000,0.005606,0.104737
KE=F,758.0,0.000811,0.021876,-0.082490,-0.012870,0.000000,0.014024,0.076244
CT=F,758.0,0.000277,0.021982,-0.272925,-0.009592,0.000655,0.011456,0.067791
ZR=F,758.0,-0.005724,0.168792,-4.615101,-0.005672,0.000000,0.006814,0.098022


In [18]:
# Compute pairwise covariance of columns
log_ret.cov()

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
GC=F,0.000123,0.000205,0.000069,0.000051,0.000023,0.000028,0.000004,0.000032,0.000028,0.000014
SI=F,0.000205,0.000548,0.000189,0.000136,0.000068,0.000065,0.000019,0.000085,0.000077,0.000023
CL=F,0.000069,0.000189,0.001630,0.000193,0.000137,0.000135,0.000021,0.000157,0.000203,0.000066
HG=F,0.000051,0.000136,0.000193,0.000265,0.000065,0.000067,0.000008,0.000062,0.000099,0.000130
LBS=F,0.000023,0.000068,0.000137,0.000065,0.001861,0.000049,0.000021,0.000051,0.000035,-0.000427
ZS=F,0.000028,0.000065,0.000135,0.000067,0.000049,0.000232,-0.000013,0.000130,0.000037,0.000075
GF=F,0.000004,0.000019,0.000021,0.000008,0.000021,-0.000013,0.000161,-0.000042,0.000009,-0.000036
KE=F,0.000032,0.000085,0.000157,0.000062,0.000051,0.000130,-0.000042,0.000479,0.000090,0.000119
CT=F,0.000028,0.000077,0.000203,0.000099,0.000035,0.000037,0.000009,0.000090,0.000483,0.000168
ZR=F,0.000014,0.000023,0.000066,0.000130,-0.000427,0.000075,-0.000036,0.000119,0.000168,0.028491


In [19]:
log_ret.cov()*252 # multiply by days

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
GC=F,0.030929,0.051567,0.017484,0.012757,0.005832,0.007103,0.001095,0.008071,0.007043,0.003614
SI=F,0.051567,0.137979,0.047541,0.034348,0.017249,0.016461,0.004706,0.021317,0.019341,0.005743
CL=F,0.017484,0.047541,0.410703,0.048571,0.034574,0.034015,0.005414,0.039569,0.051268,0.016727
HG=F,0.012757,0.034348,0.048571,0.066785,0.016297,0.016953,0.002061,0.015545,0.024956,0.032774
LBS=F,0.005832,0.017249,0.034574,0.016297,0.469006,0.012432,0.005272,0.012760,0.008935,-0.107572
ZS=F,0.007103,0.016461,0.034015,0.016953,0.012432,0.058494,-0.003261,0.032709,0.009291,0.018966
GF=F,0.001095,0.004706,0.005414,0.002061,0.005272,-0.003261,0.040492,-0.010547,0.002201,-0.008954
KE=F,0.008071,0.021317,0.039569,0.015545,0.012760,0.032709,-0.010547,0.120600,0.022751,0.029949
CT=F,0.007043,0.019341,0.051268,0.024956,0.008935,0.009291,0.002201,0.022751,0.121765,0.042222
ZR=F,0.003614,0.005743,0.016727,0.032774,-0.107572,0.018966,-0.008954,0.029949,0.042222,7.179701


In [20]:
# Set seed (optional)
np.random.seed(101)

# Stock Columns
print('portfolio_data')
print(portfolio_data.columns)
print('\n')

# Create Random Weights
print('Creating Random Weights')
weights = np.array(np.random.random(10))
print(weights)
print('\n')

# Rebalance Weights
print('Rebalance to sum to 1.0')
weights = weights / np.sum(weights)
print(weights)
print('\n')
# log_ret.mean()
# Expected Return
print('Expected Portfolio Return')
exp_ret = np.sum(log_ret.mean() * weights) *252
print(exp_ret)
print('\n')

# Expected Variance
print('Expected Volatility')
exp_vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
print(exp_vol)
print('\n')

# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio')
print(SR)

portfolio_data
Index(['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F',
       'ZR=F'],
      dtype='object')


Creating Random Weights
[0.51639863 0.57066759 0.02847423 0.17152166 0.68527698 0.83389686
 0.30696622 0.89361308 0.72154386 0.18993895]


Rebalance to sum to 1.0
[0.10499539 0.11602948 0.00578945 0.03487419 0.13933214 0.16954988
 0.0624131  0.18169153 0.14670601 0.03861884]


Expected Portfolio Return
0.04548897408333824


Expected Volatility
0.20597692643902754


Sharpe Ratio
0.2208449988538095


In [23]:
num_ports = 15000

all_weights = np.zeros((num_ports,len(portfolio_data.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    # Create Random Weights
    weights = np.array(np.random.random(10))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((log_ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

In [24]:
sharpe_arr.max()

0.933301669217653

In [25]:
sharpe_arr.argmax()

1972

In [27]:
all_weights[1972,:]

array([0.12331131, 0.02437498, 0.04899282, 0.14238339, 0.00997278,
       0.24122791, 0.1907182 , 0.18529016, 0.02888961, 0.00483883])

In [28]:
all_weights[1972,:] * equity_allocation

array([1233.11310284,  243.74983912,  489.92818988, 1423.83387689,
         99.72784338, 2412.2790975 , 1907.18204276, 1852.90162664,
        288.89608036,   48.38830064])

In [29]:
max_sr_ret = ret_arr[1972]
max_sr_vol = vol_arr[1972]

In [30]:
all_weights[1972,:]

array([0.12331131, 0.02437498, 0.04899282, 0.14238339, 0.00997278,
       0.24122791, 0.1907182 , 0.18529016, 0.02888961, 0.00483883])

In [32]:
all_weights[1972,:] * equity_allocation

array([1233.11310284,  243.74983912,  489.92818988, 1423.83387689,
         99.72784338, 2412.2790975 , 1907.18204276, 1852.90162664,
        288.89608036,   48.38830064])

In [33]:
max_sr_ret = ret_arr[1972]
max_sr_vol = vol_arr[1972]

## Plotting the data

In [34]:
scatter = hv.Scatter((vol_arr, ret_arr, sharpe_arr), 'Volatility', ['Return', 'Sharpe Ratio'])
max_sharpe = hv.Scatter([(max_sr_vol,max_sr_ret)])

scatter.opts(color='Sharpe Ratio', cmap='plasma', width=600, height=400, colorbar=True, padding=0.1) *\
max_sharpe.opts(color='red', line_color='black', size=10)

:Overlay
   .Scatter.I  :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Scatter.II :Scatter   [x]   (y)

# Mathematical Optimization

There are much better ways to find good allocation weights than just guess and check! We can use optimization functions to find the ideal weights mathematically!

### Functionalize Return and SR operations

In [35]:
def get_ret_vol_sr(weights):
    """
    Takes in weights, returns array or return,volatility, sharpe ratio
    """
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])

In [36]:
from scipy.optimize import minimize

To fully understand all the parameters, check out:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

In [37]:
#help(minimize)

Optimization works as a minimization function, since we actually want to maximize the Sharpe Ratio, we will need to turn it negative so we can minimize the negative sharpe (same as maximizing the postive sharpe)

In [38]:
def neg_sharpe(weights):
    return  get_ret_vol_sr(weights)[2] * -1

In [39]:
# Contraints
def check_sum(weights):
    '''
    Returns 0 if sum of weights is 1.0
    '''
    return np.sum(weights) - 1

In [40]:
# By convention of minimize function it should be a function that returns zero for conditions
cons = ({'type':'eq','fun': check_sum})

In [41]:
# 0-1 bounds for each weight
bounds = ((0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1))

In [42]:
# Initial Guess (equal distribution)
init_guess = [0.1,0.1,0.1,0.1, 0.1,0.1,0.1,0.1, 0.1,0.1]

In [43]:
# Sequential Least SQuares Programming (SLSQP).
opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)

In [44]:
opt_results

     fun: -1.0726845622405996
     jac: array([ 3.12924385e-07,  4.62406933e-01,  1.38044357e-04, -1.59144402e-04,
        6.70479178e-01, -2.61902809e-04,  5.07086515e-05,  4.51326370e-04,
        1.83645070e-01,  1.05712824e+01])
 message: 'Optimization terminated successfully'
    nfev: 135
     nit: 12
    njev: 12
  status: 0
 success: True
       x: array([8.31733753e-03, 0.00000000e+00, 5.27362940e-02, 1.36792367e-01,
       0.00000000e+00, 2.77004562e-01, 3.74321072e-01, 1.50828368e-01,
       0.00000000e+00, 1.73861576e-14])

In [45]:
opt_results.x

array([8.31733753e-03, 0.00000000e+00, 5.27362940e-02, 1.36792367e-01,
       0.00000000e+00, 2.77004562e-01, 3.74321072e-01, 1.50828368e-01,
       0.00000000e+00, 1.73861576e-14])

In [46]:
get_ret_vol_sr(opt_results.x)

array([0.15524216, 0.14472303, 1.07268456])

# All Optimal Portfolios (Efficient Frontier)

The efficient frontier is the set of optimal portfolios that offers the highest expected return for a defined level of risk or the lowest risk for a given level of expected return. Portfolios that lie below the efficient frontier are sub-optimal, because they do not provide enough return for the level of risk. Portfolios that cluster to the right of the efficient frontier are also sub-optimal, because they have a higher level of risk for the defined rate of return.

Efficient Frontier http://www.investopedia.com/terms/e/efficientfrontier

In [47]:
# Our returns go from 0 to somewhere along 0.3
# Create a linspace number of points to calculate x on
frontier_y = np.linspace(0,0.3,100) # Change 100 to a lower number for slower computers!

In [48]:
def minimize_volatility(weights):
    return  get_ret_vol_sr(weights)[1] 

In [49]:
frontier_volatility = []

for possible_return in frontier_y:
    # function for return
    cons = ({'type':'eq','fun': check_sum},
            {'type':'eq','fun': lambda w: get_ret_vol_sr(w)[0] - possible_return})
    
    result = minimize(minimize_volatility,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
    
    frontier_volatility.append(result['fun'])

In [50]:
scatter * hv.Curve((frontier_volatility, frontier_y)).opts(color='green', line_dash='dashed')

:Overlay
   .Scatter.I :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Curve.I   :Curve   [x]   (y)